In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-segreg.yaml')

In [3]:
# model.add_callback("on_train_start", freeze_except_regression_head)

model.train(data = '/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/yolo_dataset_nov10/data.yaml', 
            mosaic = 0.0,
            hsv_h= 0.25,  # (float) image HSV-Hue augmentation (fraction)
            hsv_s= 0.25,  # (float) image HSV-Saturation augmentation (fraction)
            hsv_v= 0.25,  # (float) image HSV-Value augmentation (fraction)
            degrees= 45.0,  # (float) image rotation (+/- deg)
            translate= 0.4,  # (float) image translation (+/- fraction)
            scale= 1.0,  # (float) image scale (+/- gain)
            #shear= 10.0,  # (float) image shear (+/- deg)
            #perspective= 0.001,  # (float) image perspective (+/- fraction), range 0-0.001
            flipud= 0.5,  # (float) image flip up-down (probability)
            fliplr= 0.5,
            epochs=100, 
            batch=1,
            reg_gain = 1.0,
            amp = False,
            warmup_epochs=0,
            imgsz=640,
            workers=12,
            lr0=3e-4,
            cos_lr = True,
            single_cls=True,
            rect=True,
            overlap_mask=False,
            mask_ratio=2,
            #freeze=5,
            #fraction=0.1,
            optimizer = "AdamW",
            #pretrained=False,
            patience=0,
            weight_decay=1e-2, 
            val=True,
            resume=True)

New https://pypi.org/project/ultralytics/8.0.215 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.166 🚀 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10018MiB)


engine/trainer: task=segment, mode=train, model=/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/runs/segment/train437/weights/last.pt, data=/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/yolo_dataset_nov10/data.yaml, epochs=100, patience=0, batch=1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=12, project=None, name=None, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=True, rect=True, cos_lr=True, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, overlap_mask=False, mask_ratio=2, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Check models for NaN weights:
import torch
for k, v in model.model.named_parameters():
    if torch.isnan(v).any():
        print(k, v)

In [ ]:
import torch.nn as nn

def get_dropout_rates(model):
    dropout_rates = []

    # Iterate over all modules (layers) in the model
    for module in model.modules():
        if isinstance(module, nn.Dropout) or \
           isinstance(module, nn.Dropout2d) or \
           isinstance(module, nn.Dropout3d):
            dropout_rates.append(module.p)

    return dropout_rates


get_dropout_rates(model.model)

In [ ]:
model.val(data = '/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/yolo_dataset_nov10/data.yaml',conf=0.25)

In [ ]:
model.val(data = '/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/yolo_dataset_nov10/data.yaml')

In [ ]:
# Show the results
from PIL import Image

for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('results.jpg')  # save image

In [ ]:
# Show the results
from PIL import Image
# Load the image
image_path = '/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/Topology-optimization-in-a-2D-femur-shaped-design-domain-Left-Classical-topology_W640.jpg'
image = Image.open(image_path)

# Print the size of the image
print(f"Image size: {image.size}")


In [ ]:
# Upscale the image by 2x:

# Upscale the image by 2x
upscale_factor = 4
upscaled_image = image.resize(
    (image.width * upscale_factor, image.height * upscale_factor),
    resample=Image.BICUBIC
)

# Print the size of the upscaled image
print(f"Upscaled image size: {upscaled_image.size}")
#show the upscaled image:
upscaled_image.show()

In [ ]:

# Process the image with your model
# results = model(image_path, conf=0.25)
results = model(upscaled_image, conf=0.15)

for r in results:
    im_array = r.plot(boxes=True,labels=False,line_width=1)  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('results.jpg')  # save image

In [ ]:
# Get a list of images path in /home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/opt_large_oct16/train/:
import os
import glob
path = "/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/yolo_dataset_nov10/valid/"

test_images = glob.glob(path + "*.png")
test_path = test_images[35].strip('.png')
#test_path = '/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/Topology-optimization-in-a-2D-femur-shaped-design-domain-Left-Classical-topology_W640.jpg'
results = model(test_path + '.png')
# results = model('/home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/yolo_dataset_nov10/valid/20231024-094956-993478.png')
results = model(upscaled_image, conf=0.25,max_det=10)

prediction_tensor = results[0].regression_preds.to('cpu').detach()
prediction_tensor

In [ ]:
#THe labels are stored in the .txt: /home/thomas/Documents/GitHub/ultralytics-custom/ultralytics/yolo_opt_final/valid/data_240.txt
import numpy as np
import torch
# Read the file line by line
with open(test_path + '.txt', 'r') as file:
    data = []
    for line in file:
        # Split the line based on whitespace and convert to float
        numbers = list(map(float, line.split()))
        
        # Take the first 7 numbers
        extracted_numbers = numbers[1:7]
        data.append(extracted_numbers)

# Convert the list to a numpy array
data_array = torch.tensor(data)
data_array

---
## Inference code:

In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
class CustomDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(CustomDiceLoss, self).__init__()
        self.size_average = size_average
    def forward(self, inputs, targets, smooth=1):
        
        # If your model contains a sigmoid or equivalent activation layer, comment this line
        #inputs = F.sigmoid(inputs)       
      
        # Check if the input tensors are of expected shape
        if inputs.shape != targets.shape:
            raise ValueError("Shape mismatch: inputs and targets must have the same shape")

        # Compute Dice loss for each sample in the batch
        dice_loss_values = []
        for input_sample, target_sample in zip(inputs, targets):
            
            # Flatten tensors for each sample
            input_sample = input_sample.view(-1)
            target_sample = target_sample.view(-1)

            intersection = (input_sample * target_sample).sum()
            dice = (2. * intersection + smooth) / (input_sample.sum() + target_sample.sum() + smooth)
            
            dice_loss_values.append(1 - dice)

        # Convert list of Dice loss values to a tensor
        dice_loss_values = torch.stack(dice_loss_values)

        # If you want the average loss over the batch to be returned
        if self.size_average:
            return dice_loss_values.mean()
        else:
            # If you want individual losses for each sample in the batch
            return dice_loss_values

def smooth_heaviside(phi, alpha, epsilon):
    # Scale and shift phi for the sigmoid function
    scaled_phi = (phi - alpha) / epsilon
    
    # Apply the sigmoid function
    H = torch.sigmoid(scaled_phi)

    return H
def calc_Phi(variable, LSgrid):
    device = variable.device  # Get the device of the variable

    x0 = variable[0]
    y0 = variable[1]
    L = variable[2]
    t1 = variable[3]
    t2 = variable[4]
    angle = variable[5]

    # Rotation
    st = torch.sin(angle)
    ct = torch.cos(angle)
    x1 = ct * (LSgrid[0][:, None].to(device) - x0) + st * (LSgrid[1][:, None].to(device) - y0) 
    y1 = -st * (LSgrid[0][:, None].to(device) - x0) + ct * (LSgrid[1][:, None].to(device) - y0)

    # Regularized hyperellipse equation
    a = L / 2  # Semi-major axis
    b = (t1 + t2) / 2  # Semi-minor axis
    small_constant = 1e-9  # To avoid division by zero
    temp = ((x1 / (a + small_constant))**6) + ((y1 / (b + small_constant))**6)

    # # Ensuring the hyperellipse shape
    allPhi = 1 - (temp + small_constant)**(1/6)

    # # Call Heaviside function with allPhi
    alpha = torch.tensor(1e-9, device=device, dtype=torch.float32)
    epsilon = torch.tensor(0.02, device=device, dtype=torch.float32)
    H_phi = smooth_heaviside(allPhi, alpha, epsilon)
    return allPhi, H_phi


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

from matplotlib.colors import TwoSlopeNorm


norm = TwoSlopeNorm(vmin=-10, vcenter=0, vmax=10)
# Load the input image
input_image_path = test_path + '.png'
input_image = Image.open(input_image_path)
input_image = upscaled_image
#I want to create a numpy array of the grayscale input image where black is 1 and white is 0:
input_image_array = np.array(input_image.convert('L'))
# input_image_array is of shape (480,640) and pred_H_resized is of shape (1,1,160,160):
# I want to calculate the dice loss between the two:
# I first need to resize the input_image_array to 160,160 using  F.interpolate
input_image_array_tensor = torch.tensor(input_image_array)

resized_input_image_array_tensor = F.interpolate(input_image_array_tensor.unsqueeze(0).unsqueeze(0), size=torch.Size([640,640]), mode='nearest')

# Normalize the resized_input_image_array_tensor:
resized_input_image_array_tensor = resized_input_image_array_tensor / 255.0

#Now the input_image has pixel values of 1.0 for white and 0.0 for black. I need to inverse that:
resized_input_image_array_tensor = 1.0 - resized_input_image_array_tensor


#I need to flip upside down the resized_input_image_array_tensor:
resized_input_image_array_tensor = torch.flip(resized_input_image_array_tensor, [2])
# Get the segmentation result
for r in results:
    im_array = r.plot(boxes=True, labels=False, line_width=1)
    seg_result = Image.fromarray(im_array[..., ::-1])



DW = 1.0
DH = 1.0


nelx = int(400 * DW)
nely = int(400 * DH)

x, y = torch.meshgrid(torch.linspace(0, DW, nelx+1), torch.linspace(0, DH, nely+1))
LSgrid = torch.stack((y.flatten(), x.flatten()), dim=0)

#xmax = torch.tensor([DW, DH, np.sqrt(DW**2 + DH**2), 0.05 * min(DW, DH), 0.05 * min(DW, DH), np.pi])
xmax = torch.tensor([1.0, 1.0, 1.0, 1.0, 0.2, 0.2])

xmin = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.01, 0.01])


xmax_preds = xmax.unsqueeze(0).expand(prediction_tensor.shape[0],-1) 
xmin_preds = xmin.unsqueeze(0).expand(prediction_tensor.shape[0],-1)

xmax = xmax.unsqueeze(0).expand(8, -1)  
xmin = xmin.unsqueeze(0).expand(8, -1) 
unnormalized_preds = prediction_tensor * (xmax_preds - xmin_preds) + xmin_preds
# # # The design variables are infered from the two endpoints and the two thicknesses:
x_center = (unnormalized_preds[:, 0] + unnormalized_preds[:, 2]) / 2
y_center = (unnormalized_preds[:, 1] + unnormalized_preds[:, 3]) / 2

L = torch.sqrt((unnormalized_preds[:, 0] - unnormalized_preds[:, 2])**2 + 
            (unnormalized_preds[:, 1] - unnormalized_preds[:, 3])**2)

L = L+1e-4
t_1 = unnormalized_preds[:, 4]
t_2 = unnormalized_preds[:, 5]

epsilon = 1e-10
y_diff = unnormalized_preds[:, 3] - unnormalized_preds[:, 1] + epsilon
x_diff = unnormalized_preds[:, 2] - unnormalized_preds[:, 0] + epsilon
theta = torch.atan2(y_diff, x_diff)
formatted_variables = torch.cat((x_center.unsqueeze(1), 
                    y_center.unsqueeze(1), 
                    L.unsqueeze(1), 
                    t_1.unsqueeze(1), 
                    t_2.unsqueeze(1), 
                    theta.unsqueeze(1)), dim=1)

pred_Phi,pred_H = calc_Phi(formatted_variables.T,LSgrid)


sum_pred_H = torch.sum(pred_H, dim=1)
sum_pred_H= torch.reshape(sum_pred_H,(1,1,nely+1,nelx+1))

# # Rearrange H_phi to the shape ([batch_size, channels, height, width])
# # Use interpolate to resize
pred_H_resized = F.interpolate(sum_pred_H, size=torch.Size([640,640]), mode='nearest')

diceloss = CustomDiceLoss()
pred_H_resized_flipped = torch.flip(pred_H_resized, [2])

#threshold both the prediction and the label with 0.5:
pred_H_resized_flipped[pred_H_resized_flipped > 0.5] = 1
pred_H_resized_flipped[pred_H_resized_flipped <= 0.5] = 0
# Calculate the dice loss:
dice_loss = diceloss(pred_H_resized_flipped, resized_input_image_array_tensor)


# Create a combined 2x2 plot
fig, axes = plt.subplots(2, 2, figsize=(12, 12)) # 2x2 subplot

# Top-left: Input image
axes[0, 0].imshow(resized_input_image_array_tensor.squeeze(),origin='lower', cmap='gray_r')
axes[0, 0].set_title('Input Image')
axes[0, 0].axis('on')

# Top-right: Segmentation Result
axes[0, 1].imshow(seg_result)
axes[0, 1].set_title('Segmentation Result')
axes[0, 1].axis('off')

# # Bottom-right: pred_Phi contour
render_colors1 = ['yellow', 'g', 'r', 'c', 'm', 'y', 'black', 'orange', 'pink', 'cyan', 'slategrey', 'wheat', 'purple', 'mediumturquoise', 'darkviolet', 'orangered']
for i, color in zip(range(0, pred_Phi.shape[1]), render_colors1*10):
    axes[1, 1].contourf(np.flipud(pred_Phi[:, i].reshape((nely+1, nelx+1))), [-0.1,0,1], colors=color)
axes[1, 1].set_title('Prediction contours')

# Bottom-left: Prediction H
axes[1, 0].imshow(pred_H_resized_flipped.squeeze().detach().numpy(), origin='lower', cmap='gray_r')
axes[1, 0].set_title('Prediction Projection')
plt.subplots_adjust(hspace=0.3, wspace=0.01)  # Adjust hspace to reduce vertical space

plt.figtext(0.5, 0.05, f'Dice Loss: {dice_loss.item():.4f}', ha='center', fontsize=16)

# Save the combined plot
fig.savefig('combined_plots.png')


In [ ]:
# input_image_array is of shape (480,640) and pred_H_resized is of shape (1,1,160,160):
# I want to calculate the dice loss between the two:
# I first need to resize the input_image_array to 160,160 using  F.interpolate
input_image_array_tensor = torch.tensor(input_image_array)


resized_input_image_array_tensor = F.interpolate(input_image_array_tensor.unsqueeze(0).unsqueeze(0), size=torch.Size([160,160]), mode='nearest')

# Normalize the resized_input_image_array_tensor:
resized_input_image_array_tensor = resized_input_image_array_tensor / 255.0

#Now the input_image has pixel values of 1.0 for white and 0.0 for black. I need to inverse that:
resized_input_image_array_tensor = 1.0 - resized_input_image_array_tensor



#I need to flip upside down the resized_input_image_array_tensor:
resized_input_image_array_tensor = torch.flip(resized_input_image_array_tensor, [2])
# Calculate the dice loss:
dice_loss = diceloss(pred_H_resized_flipped, resized_input_image_array_tensor)

dice_loss


In [ ]:
def Heaviside(phi, alpha, epsilon):
    device = phi.device  # Get the device of phi

    # For values outside of [-epsilon, epsilon]
    H_positive = torch.ones_like(phi, device=device) 
    H_negative = alpha * torch.ones_like(phi, device=device)

    # For values inside [-epsilon, epsilon]
    default = 3 * (1 - alpha) / 4 * (phi / epsilon - phi**3 / (3 * epsilon**3)) + (1 + alpha) / 2

    # Construct Heavisidve using conditions
    H = torch.where(phi > epsilon, H_positive, torch.where(phi < -epsilon, H_negative, default))

    return H


def smooth_heaviside(phi, alpha, epsilon):
    # Scale and shift phi for the sigmoid function
    scaled_phi = (phi - alpha) / epsilon
    
    # Apply the sigmoid function
    H = torch.sigmoid(scaled_phi)

    return H

# def calc_Phi(variable, LSgrid):
#     device = variable.device  # Get the device of variable
#     alpha = torch.tensor(1e-9, device=device, dtype=torch.float32)
#     epsilon = torch.tensor(0.01, device=device, dtype=torch.float32)


#     x0 = variable[0]
#     y0 = variable[1]
#     L = variable[2]
#     t1 = variable[3]
#     t2 = variable[4]
#     angle = variable[5]

#     st = torch.sin(angle)
#     ct = torch.cos(angle)

#     x1 = ct * (LSgrid[0][:, None].to(device) - x0) + st * (LSgrid[1][:, None].to(device) - y0) 
#     y1 = -st * (LSgrid[0][:, None].to(device) - x0) + ct * (LSgrid[1][:, None].to(device) - y0) 
#     small_constant = 1e-09  # You can adjust this value
#     l = (t1 + t2) / 2 + (t2 - t1) / 2 / L * x1
#     temp = ((x1)**6) / ((L**6)) + ((y1)**6) / ((l**6))
#     allPhi = 1 - (temp+small_constant)**(1/6)
    
#     # # Call Heaviside function with allPhi, alpha, and epsilon
#     H_phi = smooth_heaviside(allPhi, alpha, epsilon)

#     # return allPhi
#     return  allPhi, H_phi

def calc_Phi(variable, LSgrid):
    device = variable.device  # Get the device of the variable

    x0 = variable[0]
    y0 = variable[1]
    L = variable[2]
    t1 = variable[3]
    t2 = variable[4]
    angle = variable[5]

    # Rotation
    st = torch.sin(angle)
    ct = torch.cos(angle)
    x1 = ct * (LSgrid[0][:, None].to(device) - x0) + st * (LSgrid[1][:, None].to(device) - y0) 
    y1 = -st * (LSgrid[0][:, None].to(device) - x0) + ct * (LSgrid[1][:, None].to(device) - y0)

    # Regularized hyperellipse equation
    a = L / 2  # Semi-major axis
    b = (t1 + t2) / 2  # Semi-minor axis
    small_constant = 1e-9  # To avoid division by zero
    temp = ((x1 / (a + small_constant))**6) + ((y1 / (b + small_constant))**6)

    # # Ensuring the hyperellipse shape
    allPhi = 1 - (temp + small_constant)**(1/6)

    # # Call Heaviside function with allPhi
    alpha = torch.tensor(1e-9, device=device, dtype=torch.float32)
    epsilon = torch.tensor(0.01, device=device, dtype=torch.float32)
    H_phi = smooth_heaviside(allPhi, alpha, epsilon)
    return allPhi, H_phi






import torch
from torch.autograd import gradcheck
import numpy as np
# Your calc_Phi function here

# Preparing the grid as in your example
DW = 1.0
DH = 1.0
nelx = int(400 * DW)
nely = int(400 * DH)
x, y = torch.meshgrid(torch.linspace(0, DW, nelx+1), torch.linspace(0, DH, nely+1), indexing='xy')
LSgrid = torch.stack((y.flatten(), x.flatten()), dim=0).to(torch.float64)

# Ensure the test_tensor and p require gradients
test_tensor = torch.tensor([0.5, 0.5, 1.0, 1.0, 0.0, 0.0], requires_grad=True,dtype=torch.float64).unsqueeze(0)


# xmax = torch.tensor([1.0, 1.0, 0.75, 0.2, 0.2, np.pi])

# xmin = torch.tensor([0.0, 0.0, 0.01, 0.01, 0.00, 0.0])

xmax = torch.tensor([1.0, 1.0, 1.0, 1.0, 0.2, 0.2])

xmin = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.01, 0.01])

unnormalized_preds = test_tensor * (xmax - xmin) + xmin
# The design variables are infered from the two endpoints and the two thicknesses:
x_center = (unnormalized_preds[:, 0] + unnormalized_preds[:, 2]) / 2
y_center = (unnormalized_preds[:, 1] + unnormalized_preds[:, 3]) / 2

def safe_sqrt(x, eps=1e-6):
    return torch.sqrt(x + eps)

L = safe_sqrt((unnormalized_preds[:, 0] - unnormalized_preds[:, 2])**2 + 
            (unnormalized_preds[:, 1] - unnormalized_preds[:, 3])**2)

# gradcheck_result = gradcheck(safe_sqrt, ((unnormalized_preds[:, 0] - unnormalized_preds[:, 2])**2 + 
#             (unnormalized_preds[:, 1] - unnormalized_preds[:, 3])**2))
# print("Gradcheck passed:", gradcheck_result)

L = L+1e-4
t_1 = unnormalized_preds[:, 4]
t_2 = unnormalized_preds[:, 5]

y_diff = unnormalized_preds[:, 3] - unnormalized_preds[:, 1] 
x_diff = unnormalized_preds[:, 2] - unnormalized_preds[:, 0] 
theta = torch.atan2(y_diff, x_diff)

formatted_variables = torch.cat((x_center.unsqueeze(1), 
                    y_center.unsqueeze(1), 
                    L.unsqueeze(1), 
                    t_1.unsqueeze(1), 
                    t_2.unsqueeze(1), 
                    theta.unsqueeze(1)), dim=1)
# # # # # # # Call gradcheck
# gradcheck_result = gradcheck(calc_Phi, (unnormalized_preds.T, LSgrid))

# print("Gradcheck passed:", gradcheck_result)


In [ ]:
import matplotlib.pyplot as plt
print(unnormalized_preds)
# pred_Phi, H_phi = calc_Phi(unnormalized_preds.T, LSgrid)
pred_phi, H_phi = calc_Phi(formatted_variables.T, LSgrid)

H_phi = H_phi.detach()
import numpy as np
import matplotlib.pyplot as plt

#Calculate the sigmoid function of H_phi:
#sigmoid = 1/(1+np.exp(-H_phi))
# Plot the sigmoid function
plt.imshow(H_phi.detach().reshape((nely+1, nelx+1)), cmap='viridis', origin='lower')
plt.colorbar()



In [ ]:
import torch
import numpy as np
DW = 1.0
DH = 1.0

test_tensor = torch.tensor([0.5, 0.5, 0.5, 1.0, 1.0, 1.0]).unsqueeze(0)
nelx = int(200 * DW)
nely = int(200 * DH)

x, y = torch.meshgrid(torch.linspace(0, DW, nelx+1), torch.linspace(0, DH, nely+1))
LSgrid = torch.stack((y.flatten(), x.flatten()), dim=0)

#xmax = torch.tensor([DW, DH, np.sqrt(DW**2 + DH**2), 0.05 * min(DW, DH), 0.05 * min(DW, DH), np.pi])
xmax = torch.tensor([1.0, 1.0, 0.75, 0.05, 0.05, np.pi])

xmin = torch.tensor([0.0, 0.0, 0.001, 0.0, 0.0, 0.0])



xmax_preds = xmax.unsqueeze(0).expand(test_tensor.shape[0],-1) 
xmin_preds = xmin.unsqueeze(0).expand(test_tensor.shape[0],-1)

xmax = xmax.unsqueeze(0).expand(8, -1)  
xmin = xmin.unsqueeze(0).expand(8, -1) 

unnormalized_preds = test_tensor * (xmax_preds - xmin_preds) + xmin_preds


pred_Phi,pred_H = calc_Phi(unnormalized_preds.T,LSgrid,6,epsilon=0.2)


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(pred_H.reshape(201,201).detach().numpy())

In [ ]:
#I want to scale pred_Phi[:,0] to [0,1] and then plot it:
normalized = (pred_Phi[:,0] - pred_Phi[:,0].min()) / (pred_Phi[:,0].max() - pred_Phi[:,0].min())

plt.imshow((normalized>0.9).reshape((nely+1, nelx+1)))
plt.colorbar()


In [ ]:
F.mse_loss(normalized.reshape(201,201), pred_H.reshape(201,201))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming pred_Phi is defined somewhere in your code
# normalized = (pred_Phi[:,2] - pred_Phi[:,2].min()) / (pred_Phi[:,2].max() - pred_Phi[:,2].min())

# Create a meshgrid for plotting
x = np.linspace(0, 1, nelx+1)
y = np.linspace(0, 1, nely+1)
X, Y = np.meshgrid(x, y)

# Create a figure and 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y, np.minimum(pred_Phi[:,1],0.0).reshape((nely+1, nelx+1)), cmap='viridis')

# Add colorbar
fig.colorbar(surf)

plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Assuming you've defined and normalized your data as before...



# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(z=np.minimum(pred_Phi[:,2],0.0).reshape((nely+1, nelx+1)), x=X, y=Y, colorscale='viridis')])

fig.update_layout(title='3D Surface Plot', scene=dict(zaxis=dict(range=[-10,1])),height=800)

fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming pred_H_resized_flipped and label_H_resized are thresholded to contain values 0 or 1.

# 1. Compute the intersection
intersection = pred_H_resized_flipped * label_H_resized

# 2. Plot the original images and their intersection

# Create a combined 3x1 plot
fig, axes = plt.subplots(1, 3, figsize=(20, 12)) # 3x1 subplot to account for colorbars

# Left: label_H_resized
im0 = axes[0].imshow(label_H_resized[0,0,:,:], origin='lower')
axes[0].set_title('Ground Truth')
#fig.colorbar(im0, ax=axes[0])

# Middle: pred_H_resized_flipped
im1 = axes[1].imshow(pred_H_resized_flipped[0,0,:,:], origin='lower')
axes[1].set_title('Prediction')
#fig.colorbar(im1, ax=axes[1])

# Right: Intersection
im2 = axes[2].imshow(intersection[0,0,:,:], origin='lower')
axes[2].set_title('Intersection')
#fig.colorbar(im2, ax=axes[2])

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load the input image
input_image_path = test_path + '.png'
input_image = Image.open(input_image_path)

# Get the segmentation result
results = model(input_image_path, conf=0.25)
for r in results:
    im_array = r.plot(boxes=True, labels=False, line_width=1)
    seg_result = Image.fromarray(im_array[..., ::-1])

# Create a combined 1x2 plot
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1x2 subplot

# Left: Input image
axes[0].imshow(input_image)
axes[0].set_title('Input Image')
axes[0].axis('off')

# Right: Segmentation Result
axes[1].imshow(seg_result)
axes[1].set_title('Segmentation Result')
axes[1].axis('off')

plt.subplots_adjust(hspace=0.01)
